# Detecção de Anomalias e Machine Learning

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("deteccaoAnomaliass").getOrCreate()

In [4]:
cities_df = spark.read.format('csv').options(header = 'true', inferschema = 'true').load('D:/Downloads/csv/worldcities.csv')

In [8]:
cities_df.printSchema()

root
 |-- city: string (nullable = true)
 |-- city_ascii: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- country: string (nullable = true)
 |-- iso2: string (nullable = true)
 |-- iso3: string (nullable = true)
 |-- admin_name: string (nullable = true)
 |-- capital: string (nullable = true)
 |-- population: double (nullable = true)
 |-- id: integer (nullable = true)



In [9]:
cities_df.show()

+--------------+--------------+--------+---------+-------------+----+----+--------------------+-------+-----------+----------+
|          city|    city_ascii|     lat|      lng|      country|iso2|iso3|          admin_name|capital| population|        id|
+--------------+--------------+--------+---------+-------------+----+----+--------------------+-------+-----------+----------+
|         Tokyo|         Tokyo|  35.685| 139.7514|        Japan|  JP| JPN|               Tōkyō|primary|   3.5676E7|1392685764|
|      New York|      New York| 40.6943| -73.9249|United States|  US| USA|            New York|   null|1.9354922E7|1840034016|
|   Mexico City|   Mexico City| 19.4424|  -99.131|       Mexico|  MX| MEX|    Ciudad de México|primary|   1.9028E7|1484247881|
|        Mumbai|        Mumbai|  19.017|   72.857|        India|  IN| IND|         Mahārāshtra|  admin|   1.8978E7|1356226629|
|     São Paulo|     Sao Paulo|-23.5587|  -46.625|       Brazil|  BR| BRA|           São Paulo|  admin|   1.884

In [12]:
#filtrando algumas cidades
from pyspark.sql.functions import col
cities_BR = cities_df.where(col("country") == "Brazil")
cities_BR.show()

+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+
|          city|    city_ascii|     lat|     lng|country|iso2|iso3|         admin_name|capital|population|        id|
+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+
|     São Paulo|     Sao Paulo|-23.5587| -46.625| Brazil|  BR| BRA|          São Paulo|  admin|  1.8845E7|1076532519|
|Rio de Janeiro|Rio de Janeiro| -22.925| -43.225| Brazil|  BR| BRA|     Rio de Janeiro|  admin|  1.1748E7|1076887657|
|Belo Horizonte|Belo Horizonte| -19.915| -43.915| Brazil|  BR| BRA|       Minas Gerais|  admin| 5575000.0|1076967355|
|  Porto Alegre|  Porto Alegre|  -30.05|   -51.2| Brazil|  BR| BRA|  Rio Grande do Sul|  admin| 3917000.0|1076658221|
|      Brasília|      Brasilia|-15.7833|-47.9161| Brazil|  BR| BRA|   Distrito Federal|primary| 3716996.0|1076144436|
|        Recife|        Recife| -8.0756|-34.9156| Brazil

In [13]:
cities_MX = cities_df.where(col("country") == "Mexico")
cities_MX.show()

+--------------------+--------------------+-------+---------+-------+----+----+--------------------+-------+----------+----------+
|                city|          city_ascii|    lat|      lng|country|iso2|iso3|          admin_name|capital|population|        id|
+--------------------+--------------------+-------+---------+-------+----+----+--------------------+-------+----------+----------+
|         Mexico City|         Mexico City|19.4424|  -99.131| Mexico|  MX| MEX|    Ciudad de México|primary|  1.9028E7|1484247881|
|         Guadalajara|         Guadalajara|  20.67|  -103.33| Mexico|  MX| MEX|             Jalisco|  admin| 4198000.0|1484950208|
|           Monterrey|           Monterrey|  25.67|  -100.33| Mexico|  MX| MEX|          Nuevo León|  admin| 3712000.0|1484559591|
|              Puebla|              Puebla|  19.05|    -98.2| Mexico|  MX| MEX|              Puebla|  admin| 2195000.0|1484416394|
|             Tijuana|             Tijuana|   32.5|  -117.08| Mexico|  MX| MEX|    

In [15]:
cities_EUA = cities_df.where(col("country") == "United States")
cities_EUA.show(5)

+-----------+-----------+-------+---------+-------------+----+----+----------+-------+-----------+----------+
|       city| city_ascii|    lat|      lng|      country|iso2|iso3|admin_name|capital| population|        id|
+-----------+-----------+-------+---------+-------------+----+----+----------+-------+-----------+----------+
|   New York|   New York|40.6943| -73.9249|United States|  US| USA|  New York|   null|1.9354922E7|1840034016|
|Los Angeles|Los Angeles|34.1139|-118.4068|United States|  US| USA|California|   null|1.2815475E7|1840020491|
|    Chicago|    Chicago|41.8373| -87.6862|United States|  US| USA|  Illinois|   null|  8675982.0|1840000494|
|      Miami|      Miami|25.7839| -80.2102|United States|  US| USA|   Florida|   null|  6381966.0|1840015149|
|     Dallas|     Dallas|32.7936| -96.7662|United States|  US| USA|     Texas|   null|  5733259.0|1840019440|
+-----------+-----------+-------+---------+-------------+----+----+----------+-------+-----------+----------+
only showi

In [17]:
#criando um dataset a partir da função join (fazendo uniao)
df_concat = cities_BR.union(cities_MX)
df_concat.show()

+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+
|          city|    city_ascii|     lat|     lng|country|iso2|iso3|         admin_name|capital|population|        id|
+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+
|     São Paulo|     Sao Paulo|-23.5587| -46.625| Brazil|  BR| BRA|          São Paulo|  admin|  1.8845E7|1076532519|
|Rio de Janeiro|Rio de Janeiro| -22.925| -43.225| Brazil|  BR| BRA|     Rio de Janeiro|  admin|  1.1748E7|1076887657|
|Belo Horizonte|Belo Horizonte| -19.915| -43.915| Brazil|  BR| BRA|       Minas Gerais|  admin| 5575000.0|1076967355|
|  Porto Alegre|  Porto Alegre|  -30.05|   -51.2| Brazil|  BR| BRA|  Rio Grande do Sul|  admin| 3917000.0|1076658221|
|      Brasília|      Brasilia|-15.7833|-47.9161| Brazil|  BR| BRA|   Distrito Federal|primary| 3716996.0|1076144436|
|        Recife|        Recife| -8.0756|-34.9156| Brazil

In [19]:
#contando a quantidade de cidades em cada país
df_concat.groupby("country").count().show()

+-------+-----+
|country|count|
+-------+-----+
| Brazil|  387|
| Mexico|  188|
+-------+-----+



In [22]:
#adicionando cidades do japão (anomalias)
cities_JP = cities_df.where((col("city")=="Tokyo") | (col("city") == "Yokohama"))
cities_JP.show()

+--------+----------+------+--------+-------+----+----+----------+-------+----------+----------+
|    city|city_ascii|   lat|     lng|country|iso2|iso3|admin_name|capital|population|        id|
+--------+----------+------+--------+-------+----+----+----------+-------+----------+----------+
|   Tokyo|     Tokyo|35.685|139.7514|  Japan|  JP| JPN|     Tōkyō|primary|  3.5676E7|1392685764|
|Yokohama|  Yokohama| 35.32|  139.58|  Japan|  JP| JPN|  Kanagawa|  admin| 3697894.0|1392118339|
+--------+----------+------+--------+-------+----+----+----------+-------+----------+----------+



In [23]:
#adicionando ao dataset final
df_final = df_concat.union(cities_JP)

In [24]:
df_final.show()

+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+
|          city|    city_ascii|     lat|     lng|country|iso2|iso3|         admin_name|capital|population|        id|
+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+
|     São Paulo|     Sao Paulo|-23.5587| -46.625| Brazil|  BR| BRA|          São Paulo|  admin|  1.8845E7|1076532519|
|Rio de Janeiro|Rio de Janeiro| -22.925| -43.225| Brazil|  BR| BRA|     Rio de Janeiro|  admin|  1.1748E7|1076887657|
|Belo Horizonte|Belo Horizonte| -19.915| -43.915| Brazil|  BR| BRA|       Minas Gerais|  admin| 5575000.0|1076967355|
|  Porto Alegre|  Porto Alegre|  -30.05|   -51.2| Brazil|  BR| BRA|  Rio Grande do Sul|  admin| 3917000.0|1076658221|
|      Brasília|      Brasilia|-15.7833|-47.9161| Brazil|  BR| BRA|   Distrito Federal|primary| 3716996.0|1076144436|
|        Recife|        Recife| -8.0756|-34.9156| Brazil

In [25]:
from pyspark.ml.evaluation import ClusteringEvaluator #biblioteca utilizada para a avaliação em cada um dos cluters
from pyspark.ml.clustering import KMeans

In [26]:
from pyspark.ml.feature import VectorAssembler #transoforma os dados em vetores de característica

vecAssembler = VectorAssembler(inputCols = ['lat','lng'], outputCol = 'features') #utilizada para transoformar os dados em vetor
new_df = vecAssembler.transform(df_final) #aplica a transformação
new_df.show()

+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+-------------------+
|          city|    city_ascii|     lat|     lng|country|iso2|iso3|         admin_name|capital|population|        id|           features|
+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+-------------------+
|     São Paulo|     Sao Paulo|-23.5587| -46.625| Brazil|  BR| BRA|          São Paulo|  admin|  1.8845E7|1076532519| [-23.5587,-46.625]|
|Rio de Janeiro|Rio de Janeiro| -22.925| -43.225| Brazil|  BR| BRA|     Rio de Janeiro|  admin|  1.1748E7|1076887657|  [-22.925,-43.225]|
|Belo Horizonte|Belo Horizonte| -19.915| -43.915| Brazil|  BR| BRA|       Minas Gerais|  admin| 5575000.0|1076967355|  [-19.915,-43.915]|
|  Porto Alegre|  Porto Alegre|  -30.05|   -51.2| Brazil|  BR| BRA|  Rio Grande do Sul|  admin| 3917000.0|1076658221|     [-30.05,-51.2]|
|      Brasília|      Brasilia|-15

In [27]:
#aplica o processo de clusterização
kmeans = KMeans(k=3, seed = 1)
model = kmeans.fit(new_df.select('features')) #aplica o treinamento

In [28]:
#cria o dataset com a indicação sobre qual cluster cada conjunto de dados foi adicionado
df_final = model.transform(new_df)
df_final.show()

+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+-------------------+----------+
|          city|    city_ascii|     lat|     lng|country|iso2|iso3|         admin_name|capital|population|        id|           features|prediction|
+--------------+--------------+--------+--------+-------+----+----+-------------------+-------+----------+----------+-------------------+----------+
|     São Paulo|     Sao Paulo|-23.5587| -46.625| Brazil|  BR| BRA|          São Paulo|  admin|  1.8845E7|1076532519| [-23.5587,-46.625]|         0|
|Rio de Janeiro|Rio de Janeiro| -22.925| -43.225| Brazil|  BR| BRA|     Rio de Janeiro|  admin|  1.1748E7|1076887657|  [-22.925,-43.225]|         0|
|Belo Horizonte|Belo Horizonte| -19.915| -43.915| Brazil|  BR| BRA|       Minas Gerais|  admin| 5575000.0|1076967355|  [-19.915,-43.915]|         0|
|  Porto Alegre|  Porto Alegre|  -30.05|   -51.2| Brazil|  BR| BRA|  Rio Grande do Sul|  admin| 3917000.0|

In [29]:
df_final.groupby("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         0|  387|
|         1|  188|
|         2|    2|
+----------+-----+



In [30]:
df_final.where(col('prediction')=='2').show()

+--------+----------+------+--------+-------+----+----+----------+-------+----------+----------+-----------------+----------+
|    city|city_ascii|   lat|     lng|country|iso2|iso3|admin_name|capital|population|        id|         features|prediction|
+--------+----------+------+--------+-------+----+----+----------+-------+----------+----------+-----------------+----------+
|   Tokyo|     Tokyo|35.685|139.7514|  Japan|  JP| JPN|     Tōkyō|primary|  3.5676E7|1392685764|[35.685,139.7514]|         2|
|Yokohama|  Yokohama| 35.32|  139.58|  Japan|  JP| JPN|  Kanagawa|  admin| 3697894.0|1392118339|   [35.32,139.58]|         2|
+--------+----------+------+--------+-------+----+----+----------+-------+----------+----------+-----------------+----------+

